###引入模組、定義參數、連接雲端硬碟



In [1]:
# 引入模組
import numpy as np
import pandas as pd
import openpyxl

# 檔案、資料夾路徑參數
gdrive_path = '/content/drive' # 雲端硬碟路徑
project_folder = '/content/drive/MyDrive/Inundation_Forcasting/' # 專案資料夾的路徑
output_folder = '/content/drive/MyDrive/Inundation_Forcasting/outputs/' # 輸出結果的資料夾路徑

# 要計算的指標
index_names = ['RMSE', 'MAE', 'CE', 'CC', 'EQp', 'ETp']

# 連接雲端硬碟
from google.colab import drive
drive.mount(gdrive_path)


Mounted at /content/drive


###讀取資料

In [2]:
# 載入資料
def load_data(path):
  wb = openpyxl.load_workbook(path, data_only=True)
  data = {}
  for i in range(len(wb.sheetnames)):
    df = pd.DataFrame(get_values(wb[wb.sheetnames[i]]))
    df.columns = df.iloc[0]
    df = df.drop([0], axis=0)
    data[wb.sheetnames[i]] = df
  return data

# 取得資料值並儲存為列表
def get_values(sheet):
  arr = [] 
  for row in sheet:
    temp = []  
    for column in row:
        temp.append(column.value)
    arr.append(temp)
  return arr

# 讀取資料
RES_train = load_data(output_folder + 'RES-train.xlsx')
RES_test = load_data(output_folder + 'RES-test.xlsx')

# 事件數量
num_events = len(RES_train) - 2

# 用 RES-train['工作表名稱']['資料欄位'] 來取得所需資料
All_Estimation = RES_test['RES-test_EVall']['Estimation']
print(All_Estimation)


1       8.03
2       7.01
3       6.92
4       6.58
5       7.78
       ...  
316    93.76
317    93.42
318    92.56
319    92.48
320    96.45
Name: Estimation, Length: 320, dtype: object


###計算各項指標

In [3]:
index_train = {}
index_test = {}

for i in range(num_events):
  event_train = "RES-train_EV" + "%02d" %(i+1)
  event_test = "RES-test_EV" + "%02d" %(i+1)

  # 取得訓練資料及測試資料的實際值(Observation)和預測值(Estimation)
  obv_train = np.array(RES_train[event_train]['Observation'], dtype=np.float64)
  est_train = np.array(RES_train[event_train]['Estimation'], dtype=np.float64)
  obv_test = np.array(RES_test[event_test]['Observation'], dtype=np.float64)
  est_test = np.array(RES_test[event_test]['Estimation'], dtype=np.float64)

  # 計算訓練資料的指標
  rmse_train = np.sqrt(np.mean((est_train - obv_train) ** 2))
  mae_train = np.mean(np.abs(est_train - obv_train))
  ce_train = 1 - np.sum((est_train - obv_train) ** 2) / np.sum((obv_train - np.mean(obv_train)) ** 2)
  cc_train = np.corrcoef(est_train, obv_train)[0, 1]
  eqp_train = (np.max(est_train) - np.max(obv_train)) / np.max(obv_train)
  etp_train = np.abs(np.argmax(est_train) - np.argmax(obv_train))

  # 計算測試資料的指標
  rmse_test = np.sqrt(np.mean((est_test - obv_test) ** 2))
  mae_test = np.mean(np.abs(est_test - obv_test))
  ce_test = 1 - np.sum((est_test - obv_test) ** 2) / np.sum((obv_test - np.mean(obv_test)) ** 2)
  cc_test = np.corrcoef(est_test, obv_test)[0, 1]
  eqp_test = (np.max(est_test) - np.max(obv_test)) / np.max(obv_test)
  etp_test = np.abs(np.argmax(est_test) - np.argmax(obv_test))

  # 儲存計算後的指標在字典中
  index_train[event_train] = {'RMSE':rmse_train, 'MAE':mae_train, 'CE':ce_train, 'CC':cc_train, 'EQp':eqp_train, 'ETp':etp_train}
  index_test[event_test] = {'RMSE':rmse_test, 'MAE':mae_test, 'CE':ce_test, 'CC':cc_test, 'EQp':eqp_test, 'ETp':etp_test}

print(index_train)
print(index_test)


{'RES-train_EV01': {'RMSE': 18.931932628415804, 'MAE': 13.078398437499999, 'CE': 0.8483332425445358, 'CC': 0.927342020900491, 'EQp': -0.07710323574730349, 'ETp': 1}, 'RES-train_EV02': {'RMSE': 13.627826770985902, 'MAE': 8.87203125, 'CE': 0.9278742296657064, 'CC': 0.9658134163179837, 'EQp': -0.07395993836671803, 'ETp': 1}, 'RES-train_EV03': {'RMSE': 16.578690471034648, 'MAE': 10.9480859375, 'CE': 0.881202940580859, 'CC': 0.9476342425918606, 'EQp': -0.19573003722171495, 'ETp': 63}, 'RES-train_EV04': {'RMSE': 16.729094687152802, 'MAE': 10.83978448275862, 'CE': 0.890022077565101, 'CC': 0.9470563924701538, 'EQp': -0.2096147919876733, 'ETp': 175}, 'RES-train_EV05': {'RMSE': 18.371386972292385, 'MAE': 13.61357142857143, 'CE': 0.8550859931162674, 'CC': 0.9359217466846831, 'EQp': -0.08246533127889057, 'ETp': 1}}
{'RES-test_EV01': {'RMSE': 15.559872217903976, 'MAE': 12.66078125, 'CE': 0.9095021352114832, 'CC': 0.9656604745542963, 'EQp': -0.21567525809396715, 'ETp': 10}, 'RES-test_EV02': {'RMSE':

###將指標寫入excel檔

In [4]:
wb = openpyxl.Workbook()
del wb['Sheet']

# 將不同指標寫入不同工作表
for index_name in index_names:
  sheet = wb.create_sheet(index_name)
  sheet.append([ 'Event', 'Train', 'Test' ])
  for i in range(num_events):
    event_train = "RES-train_EV" + "%02d" %(i+1)
    event_test = "RES-test_EV" + "%02d" %(i+1)
    sheet.append([ i+1, round(index_train[event_train][index_name], 4), round(index_test[event_test][index_name], 4) ])

# 輸出excel檔
path = output_folder + 'Index.xlsx'
wb.save(path)
print("Result saved in " + path)


Result saved in /content/drive/MyDrive/Inundation_Forcasting/outputs/Index.xlsx
